In [61]:
import jax
import jax.numpy as jnp
import numpy as np

In [73]:
class Node:
    def __init__(self, bias, in_weights, out_weights, id):
        """
        Setup for simple node
        We get inputs from other nodes and outputs to other nodes
        The nummber and connection nodes in training should dynamically change
        The processing inside is not fixed and will be either an embedding or attention
        To close mimic real neurons the activation will also have a deacay time

        This architecture is still in development only some features will be added for now
        so debugging is simmpler

        The inputs aggregate into a single state.
        """
        # setup
        self.bias = bias # float
        self.in_weights = in_weights # float tensor
        self.out_weights = out_weights # float tensor

        # processing
        # Here development will happen
        self.activation = 0 # float

        # used in other classes
        self.id = id
    
# Run functions
    def run(self, inputs):
        # processing inputs and computing next step
        self.activation = jnp.dot(self.in_weights, inputs) + self.bias
        outputs = jnp.dot(self.out_weights, self.activation)
        return outputs
    
# Topology functions
    # TODO: Change so that it's connection wise
    # TODO: Check if functions work
    def add_input(self, weight):
        #Add a new input to the node
        self.in_weights = jnp.append(self.in_weights, weight)
        return None
    
    def add_output(self, weight):
        #Add a new input to the node
        self.out_weights = jnp.append(self.out_weights, weight)
        return None
    
    def remove_input(self, index):
        #Remove a input from the node
        self.in_weights = jnp.delete(self.in_weights, index)
        return None
    
    def remove_output(self, index):
        # Remove a output from the node
        self.out_weights = jnp.delete(self.out_weights, index)
        return None

# Get functions
    def get_id(self):
        return self.id

# Debugging functions
    def get_activation(self):
        return self.activation 
    
    def get_bias(self):
        return self.bias
    
    def get_in_weights(self):
        return self.in_weights
    
    def get_out_weights(self):
        return self.out_weights
    
    def get_input_size(self):
        return self.in_weights.size
    
    def get_output_size(self):
        return self.out_weights.size
    
    def print_node(self):
        # Gives a overview of the node
        print("Node id: ", self.id)
        print("Activation: ", self.activation)
        print("Bias: ", self.bias)
        print("Inputs: ", self.in_weights)
        print("Outputs: ", self.out_weights)
        return None

In [79]:
# create a node
# 3 inputs, 3 outputs
Node0 = Node(0, jnp.array([0.1, 0.2, 0.3]), jnp.array([0.1, 0.2, 0.3]), 0)
# 3 inputs, 1 outputs
Node1 = Node(0, jnp.array([0.1, 0.2, 0.3]), jnp.array([0.1]), 1)


In [80]:
Node0.print_node()
Node0.remove_input(0)
Node0.print_node()
Node0.add_output(0.5)
Node0.print_node()

Node id:  0
Bias:  0
Inputs:  [0.1 0.2 0.3]
Outputs:  [0.1 0.2 0.3]
Activation:  0
Node id:  0
Bias:  0
Inputs:  [0.2 0.3]
Outputs:  [0.1 0.2 0.3]
Activation:  0
Node id:  0
Bias:  0
Inputs:  [0.2 0.3]
Outputs:  [0.1 0.2 0.3 0.5]
Activation:  0


In [64]:
# Test for node
x = jnp.array([1, 2, 3])
print(Node0.processing(x))
print(Node1.processing(x))

[0.14000002 0.28000003 0.42000005]
[0.14000002]


In [65]:
class ComputationTree:
    def __init__(self, nodes: list[Node] = None):
        """
        This class saves and edits the computation tree of the nodes
        Meaning it saves the connections between the nodes
        Additionally it displays neighboring nodes needed for training
        """
        # TODO : Code for when no nodes are given
        # TODO : Make compatible with node ids
        self.nodes = nodes # jnp.array of nodes
        self.connections = jnp.zeros((len(nodes), len(nodes))) # directional adjecency matrix, row 

    def add_node(self, node):
        #TODO: Check
        """
        Add a new node to the computation tree
        """
        self.nodes.append(node)
        # not connected to any node
        self.connections = jnp.append(self.connections, jnp.zeros((len(self.connections), 1)), axis=1)
        self.connections = jnp.append(self.connections, jnp.zeros((1, len(self.connections[0]))), axis=0)
        return None
    
    def remove_node(self, index):
        self.nodes.delete(index)
        self.connections = jnp.delete(self.connections, index, axis=0)
        self.connections = jnp.delete(self.connections, index, axis=1)
        return None

    def add_connection(self, node1, node2):
        # get ids like addresses in connection
        node1_id = int(node1.get_id())
        node2_id = int(node2.get_id())

        # add connection
        self.connections = self.connections.at[node1_id, node2_id].set(1)
        return None
    
    def get_neighbors(self, node, degree):
        # get the neighbor of a certain degree of a node
        # degree 1: direct neighbors, degree 2: neighbors of neighbors, ...

        # init empty list to collect neighbors
        neighbors = []

        # recursive search
        if degree == 1:
            neighbors = list(jnp.where(self.connections[node.get_id()] == 1)[0])
        else:
            cur_neighbors = list(jnp.where(self.connections[node.get_id()] == 1)[0])
            # go over neighbors
            for cur_neighbor in cur_neighbors:
                neighbors.extend(self.get_neighbors(self.nodes[cur_neighbor], degree - 1))
        return neighbors
        
    # debugging functions
    def print_connections(self):
        print(self.connections)
        return None

In [66]:
# Create a few more nodes for testing
Node2 = Node(0, jnp.array([0.1, 0.2, 0.3]), jnp.array([0.1, 0.2, 0.3]), 2)
Node3 = Node(0, jnp.array([0.1, 0.2, 0.3]), jnp.array([0.1, 0.2, 0.3]), 3)
Node4 = Node(0, jnp.array([0.1, 0.2, 0.3]), jnp.array([0.1, 0.2, 0.3]), 4)

In [67]:
# Test prep for ComputationTree
nodes = [Node0, Node1, Node2, Node3, Node4] # used nodes
CT = ComputationTree(nodes)
CT.add_connection(Node0, Node1)
CT.add_connection(Node1, Node2)
CT.add_connection(Node0, Node3)
CT.add_connection(Node3, Node4)

In [71]:
# Test ComputationTree
CT.print_connections()
print(CT.get_neighbors(Node0, 2))

[[0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]
[Array(2, dtype=int32), Array(4, dtype=int32)]


In [69]:
# resume development after compuleting computation Tree
class Cluster:
    def __init__(self, input, outputs, num_nodes):
        """
        This creates a cluster of nodes
        The cluster will have an input and output
        these are either connected to other clusters or inference for the user
        """

    def processing(self, inputs):
        """
        Run one cycle with all nodes
        """
        pass

    def expand(self):
        """
        Expands the Cluster depending on information density
        and the surrounding nodes
        """
        pass

    def connections(self):
        pass
    
    def get_nodes(self):
        pass
